<a href="https://colab.research.google.com/github/technoavengers/Apache_Beam/blob/main/Assignment3_Sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install apache_beam

In [12]:
import apache_beam as beam
p1 = beam.Pipeline()

side_list = list()
with open ('credit_card_exclude.txt','r') as exclude_file:
  for cust_id in exclude_file:
    side_list.append(cust_id.rstrip())

print(side_list)

class ProcessCustomers(beam.DoFn):
  def process(self,element,side_list):
    record = element.split(',')
    if record[0] not in side_list:
      if(int(record[5])-int(record[6]) <=100):
        yield record[0]
      if(record[4]==('Visa')):
        yield  beam.pvalue.TaggedOutput('Visa_Holders',element)


customers = (
    p1
    |beam.io.ReadFromText('credit_cards.txt',skip_header_lines=1)
    |beam.ParDo(ProcessCustomers(),side_list).with_outputs('Visa_Holders', main='Max_Credit_Limit')
)

visa_holder = customers.Visa_Holders
max_credit_limit = customers.Max_Credit_Limit

visa_holder | 'Write Visa Holder Customers' >> beam.io.WriteToText("visa-holder")
max_credit_limit  | beam.Distinct() | 'Write Customers who have used their Max Credit Limit' >> beam.io.WriteToText("max_credit_limit")


p1.run()

['CT65528', 'CT62963', 'CT69061', 'CT27825']


In [ ]:
! cat /content/visa-holder-00000-of-00001

In [ ]:
! cat /content/max_credit_limit-00000-of-00001